TWilson July 2018 **F1 Race Car Image Detector using TensorFlow Object Detector API**
help in doing this was taken from these blogs:
https://medium.com/@moshe.livne/training-tensorflow-for-free-pet-object-detection-api-sample-trained-on-google-collab-c2e65f4a9949 

https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/ 

http://andrew.carterlunn.co.uk/programming/2018/01/24/annotating-large-datasets-with-tensorflow-object-detection-api.html 

Make sure runtime is set to use GPU:

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

 Install pre-requisites for object detection:

In [0]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml


In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
import os
os.chdir('models/research')

In [0]:

!protoc object_detection/protos/*.proto --python_out=.

Set paths for Object Detection API scripts:

In [0]:
import sys
sys.path.append('/content/models/research/slim')
sys.path.append('/content/models/research/object_detection')

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

In [0]:
%run object_detection/builders/model_builder_test.py

Upload in turn - F1 training TFRecord file, validation TFRecord file, label map text file, SSD config file:

In [0]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Move these items to relevant directories:

In [0]:
!cp pascal_label_map.pbtxt object_detection/data/

In [0]:
!cp f1train.record object_detection/data/
!cp f1valid.record object_detection/data/

In [0]:

!mkdir object_detection/training
!cp ssd_mobilenet_v1_coco.config object_detection/training/ssd_mobilenet_v1_coco.config

Get the pre-trained model checkpoint, ssd_mobilenet_v1_coco_2017_11_17:

In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2017_11_17.tar.gz

Untar and move this to Object detection directory:

In [0]:
!tar -xzvf ssd_mobilenet_v1_coco_2017_11_17.tar.gz

In [0]:
!mv ssd_mobilenet_v1_coco_2017_11_17 object_detection/ssd_mobilenet_v1_coco_2017_11_17

**Optional:** setup TensorFlow and Run scripts in background, or skip this and move to step labelled RUN:

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

In [0]:
mv ngrok object_detection/ngrok

In [0]:
os.chdir('object_detection')

In [0]:
!mkdir evaluation

In [0]:
LOG_DIR = './evaluation'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
get_ipython().system_raw('python train.py \
--logtostderr \
--pipeline_config_path=training/ssd_mobilenet_v1_coco.config \
--checkpoint_dir=training \
--train_dir=training/ 2>&1 /content/log/tb.log &')


In [0]:
get_ipython().system_raw('python eval.py \
--logtostderr \
--pipeline_config_path=training/ssd_mobilenet_v1_coco.config \
--checkpoint_dir=training \
--eval_dir=evaluation 2>&1 eval.log')

**RUN:** if not using TensorBoard, best to not run scripts in background, but in foreground, so can see progress:

In [0]:
os.chdir('object_detection')
%run train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_coco.config

Once run will need to delete flags before can either run again, or export graph:

In [0]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
del_all_flags(tf.flags.FLAGS)

Make sure the trained graph is in the directory:

In [0]:
!ls training

Make a directory to copy the exported graph:

In [0]:
!mkdir exported_graph

Zip the training directory, if wish to download this as well as just the exported graph:

In [0]:
!zip -r training_done.zip training

Export the graph:

In [0]:
%run export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=training/ssd_mobilenet_v1_coco.config \
    --trained_checkpoint_prefix=training/model.ckpt-5000 \
    --output_directory=exported_graph

Zip exported graph for download:

In [0]:
!zip -r exp_g.zip exported_graph

Set up connection to Google Drive in order to download the graph:

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Set the name of the drive connection as a folder named "drive"

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

Copy the zipped exported graph and the zipped train directory to Google Drive so can be downloaded

In [0]:
!cp exp_g.zip drive/colab_data

In [0]:
!cp training_done.zip drive/colab_data